Implementation of Connect 4 AI with inspirations from the following:
    http://blog.gamesolver.org/solving-connect-four/01-introduction/
    https://github.com/nwestbury/pyConnect4
    https://towardsdatascience.com/creating-the-perfect-connect-four-ai-bot-c165115557b0


In [2]:
import numpy
from kaggle_environments import make

# env = make("connectx", debug=True)
# # None indicates which agent will be manually played.
# env.run(["random", "random"])
# env.render(mode="ipython", width=500, height=450)

In [2]:
# from submission import my_agent

In [3]:
env = make("connectx", debug=True)
# None indicates which agent will be manually played.
# env.run(["negamax", "negamax"])
# env.render(mode="ipython", width=500, height=450)

In [4]:
def my_agent3(obs, config):
    import numpy as np
    import random
    import time
    
    
    # Gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(config.rows-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

    # Helper function for get_heuristic: checks if window satisfies heuristic conditions
    def check_window(window, num_discs, piece, config):
        return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)

    # Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
    def count_windows(grid, num_discs, piece, config):
        num_windows = 0
        # horizontal
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        return num_windows
    
    # Helper function for minimax: calculates value of heuristic for grid
    def get_heuristic(grid, mark, config):
        num_threes = count_windows(grid, 3, mark, config)
        num_fours = count_windows(grid, 4, mark, config)
        num_threes_opp = count_windows(grid, 3, mark%2+1, config)
        num_fours_opp = count_windows(grid, 4, mark%2+1, config)        
#         score = 1e2*num_threes - 1e2*num_threes_opp - 1e9*num_fours_opp + 1e9*num_fours
        score = 10 * num_threes - 1e2*num_threes_opp - 1e4*num_fours_opp + 1e6*num_fours
        return score
    
    # Uses minimax to calculate value of dropping piece in selected column
    def score_move(grid, col, mark, config, nsteps):
        next_grid = drop_piece(grid, col, mark, config)        
        score = -negamax(next_grid, nsteps - 1, mark%2+1, config)
        return score

    # Helper function for minimax: checks if agent or opponent has four in a row in the window
    def is_terminal_window(window, config):
        return window.count(1) == config.inarow or window.count(2) == config.inarow

    # Helper function for minimax: checks if game has ended
    def is_terminal_node(grid, config):
        # Check for draw 
        if list(grid[0, :]).count(0) == 0:
            return True
        
        # Check for win: horizontal, vertical, or diagonal
        # horizontal 
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                if is_terminal_window(window, config):
                    return True
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                if is_terminal_window(window, config):
                    return True
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if is_terminal_window(window, config):
                    return True
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if is_terminal_window(window, config):
                    return True
        return False
    

    def negamax(node, depth, mark, config, alpha=-np.Inf, beta=np.Inf):
        """
        function negamax(node, depth, α, β, color) is
        if depth = 0 or node is a terminal node then
            return color × the heuristic value of node

        childNodes := generateMoves(node)
        childNodes := orderMoves(childNodes)
        value := −∞
        foreach child in childNodes do
            value := max(value, −negamax(child, depth − 1, −β, −α, −color))
            α := max(α, value)
            if α ≥ β then
                break (* cut-off *)
        return alpha
        """
        global pos_count
        pos_count += 1
        is_terminal = is_terminal_node(node, config)
        
        if depth == 0 or is_terminal:
            return get_heuristic(node, mark, config)
#         valid_moves = [c for c in range(config.columns) if node[0][c] == 0]  
        valid_moves = [column_order[col] for col in range(config.columns) if obs.board[column_order[col]] == 0]
#         valid_moves := orderMoves(valid_moves)
        value = -np.Inf
        for col in valid_moves:
            child = drop_piece(node, col, mark, config)
            value = max(value, -negamax(child, depth - 1, mark%2+1, config, -beta, -alpha))
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return alpha
    
    
#     def iterative_deepening(grid, obs.mark, config, timeout):
#         best_col, best_score = -1, -np.Inf
#         valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
#         for depth in range(3, 50):
#             for col in valid_moves:
#                 next_grid = drop_piece(grid, col, mark, config)        
#                 score = -negamax(next_grid, nsteps - 1, mark%2+1, config)
#                 if not search_cutoff and score > best_score:
#                     best_col, best_score = col, best_score
#                 else: # Running out of time
#                     break
#         return best_col
        
        
        
        
  
    start = time.time()
    global pos_count
    pos_count = 0
    column_order = []
    for i in range(7):
        column_order.append(7//2 + (1 - 2* (i %2)) * (i+1)//2)
    
#     valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    # Order column from center to side
    valid_moves = [column_order[col] for col in range(config.columns) if obs.board[column_order[col]] == 0]
    # Very first move, always play the middle
    if sum(obs.board) == 0:
        return 3
    elif sum(obs.board) == 1 and 3 in valid_moves:
        return 3 # Play middle whether opponent places there or not
    
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # print(grid)
    #Get scores for each branch
    N_STEPS = 4
    
    # Use the heuristic to assign a score to each possible board in the next step
    if len(valid_moves) >= 6:
        N_STEPS=4
    if len(valid_moves) ==5:
        N_STEPS=5
    if len(valid_moves) ==4:
        N_STEPS=6
    if len(valid_moves) <=3:
        N_STEPS=10
        
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))        
    
    #Get the highest score value    
    
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]    
    col = max_cols[0] # Taking the center columns first
    end = time.time()
#     print("timeout", config.timeout)
    print("my_agent3 excecution time", (end-start), "move", col, "score", scores[col], "at depth", N_STEPS, "pos count", pos_count)
    
    return col

        

In [15]:
import numpy as np
from functools import lru_cache

class Position(object):
    WIDTH = 7
    HEIGHT = 6
    def __init__(self, position, mask):
        self.position = position
        self.mask = mask

    @classmethod
    def get_position_mask_bitmap(self, grid, mark):
        # print("grid", grid)
        # print("mark", mark)
        position, mask = '', ''
        # Start with right-most column
        for j in range(6, -1, -1):
            # Add 0-bits to sentinel 
            mask += '0'
            position += '0'
            # Start with bottom row
            for i in range(0, 6):
                mask += ['0', '1'][int(grid[i][j] != 0)]
                position += ['0', '1'][int(grid[i][j] == mark)]
        return int(position, 2), int(mask, 2)
    
    def has_won(self, position):
        # Horizontal check
        m = position & (position >> 7)
        if m & (m >> 14):
            return True
        # Diagonal \
        m = position & (position >> 6)
        if m & (m >> 12):
            return True
        # Diagonal /
        m = position & (position >> 8)
        if m & (m >> 16):
            return True
        # Vertical
        m = position & (position >> 1)
        if m & (m >> 2):
            return True
        # Nothing found
        return False

    def can_play(self, col):
        return (self.mask & self.top_mask(col)) == 0

    def play(self, col):
        self.position ^= self.mask
        self.mask |= self.mask + self.bottom_mask(col)

    def is_winning_move(self, col):
        pos = self.position
        pos |= (self.mask + self.bottom_mask(col)) & self.column_mask(col)
        return self.has_won(pos)
    
    def has_drawn(self):
        """
        If the board has all of its valid slots filled, then it is a draw.
        We mask the board to a bitboard with all positions filled
        (0xFDFBF7EFDFBF) and if all the bits are active, it is a draw.
        """
        return (self.position & 0xFDFBF7EFDFBF) == 0xFDFBF7EFDFBF

    
    # return a bitmask containg a single 1 corresponding to the top cel of a given column
    def top_mask(self, col):
        return (1 << (self.HEIGHT - 1)) << col * (self.HEIGHT + 1)

    # return a bitmask containg a single 1 corresponding to the bottom cell of a given column
    def bottom_mask(self, col):
        return 1 << col * (self.HEIGHT + 1)

    #return a bitmask 1 on all the cells of a given column
    def column_mask(self, col):
        return ((1 << self.HEIGHT) - 1) << col * (self.HEIGHT + 1)

    def pretty_print(self):
        opp_position = self.position ^ self.mask
        print("board     position  mask      key       bottom")
        print("          0000000   0000000")
        for i in range(5, -1, -1): # iterate backwards from 5 to 0
            board_row = "".join("x" if (self.position >> i+k*7) & 1 == 1 \
                else "o" if (opp_position >> i+k*7) & 1 == 1 else "." for k in range(8))
            pos_row = "".join(str((self.position >> i+j*7) & 1) for j in range(7))
            mask_row = "".join(str((self.mask >> i+j*7) & 1) for j in range(7))
            print(board_row + "   " + pos_row + "   " + mask_row)
    
    def evaluate3(self, oppBoard, myBoard):
        """
        Returns the number of possible 3 in a rows in bitboard format.
        Running time: O(1)
        http://www.gamedev.net/topic/596955-trying-bit-boards-for-connect-4/
        """
        inverseBoard = ~(myBoard | oppBoard)
        rShift7MyBoard = myBoard >> 7
        lShift7MyBoard = myBoard << 7
        rShift14MyBoard = myBoard >> 14
        lShit14MyBoard = myBoard << 14
        rShift16MyBoard = myBoard >> 16
        lShift16MyBoard = myBoard << 16
        rShift8MyBoard = myBoard >> 8
        lShift8MyBoard = myBoard << 8
        rShift6MyBoard = myBoard >> 6
        lShift6MyBoard = myBoard << 6
        rShift12MyBoard = myBoard >> 12
        lShift12MyBoard = myBoard << 12

        # check _XXX and XXX_ horizontal
        result = inverseBoard & rShift7MyBoard & rShift14MyBoard\
            & (myBoard >> 21)

        result |= inverseBoard & rShift7MyBoard & rShift14MyBoard\
            & lShift7MyBoard

        result |= inverseBoard & rShift7MyBoard & lShift7MyBoard\
            & lShit14MyBoard

        result |= inverseBoard & lShift7MyBoard & lShit14MyBoard\
            & (myBoard << 21)

        # check XXX_ diagonal /
        result |= inverseBoard & rShift8MyBoard & rShift16MyBoard\
            & (myBoard >> 24)

        result |= inverseBoard & rShift8MyBoard & rShift16MyBoard\
            & lShift8MyBoard

        result |= inverseBoard & rShift8MyBoard & lShift8MyBoard\
            & lShift16MyBoard

        result |= inverseBoard & lShift8MyBoard & lShift16MyBoard\
            & (myBoard << 24)

        # check _XXX diagonal \
        result |= inverseBoard & rShift6MyBoard & rShift12MyBoard\
            & (myBoard >> 18)

        result |= inverseBoard & rShift6MyBoard & rShift12MyBoard\
            & lShift6MyBoard

        result |= inverseBoard & rShift6MyBoard & lShift6MyBoard\
            & lShift12MyBoard

        result |= inverseBoard & lShift6MyBoard & lShift12MyBoard\
            & (myBoard << 18)

        # check for _XXX vertical
        result |= inverseBoard & (myBoard << 1) & (myBoard << 2)\
            & (myBoard << 3)

        return result

    def evaluate2(self, myBoard, oppBoard):
        """
        Returns the number of possible 2 in a rows in bitboard format.
        Running time: O(1)
        """
        inverseBoard = ~(myBoard | oppBoard)
        rShift7MyBoard = myBoard >> 7
        rShift14MyBoard = myBoard >> 14
        lShift7MyBoard = myBoard << 7
        lShift14MyBoard = myBoard << 14
        rShift8MyBoard = myBoard >> 8
        lShift8MyBoard = myBoard << 8
        lShift16MyBoard = myBoard << 16
        rShift16MyBoard = myBoard >> 16
        rShift6MyBoard = myBoard >> 6
        lShift6MyBoard = myBoard << 6
        rShift12MyBoard = myBoard >> 12
        lShift12MyBoard = myBoard << 12

        # check for _XX
        result = inverseBoard & rShift7MyBoard & rShift14MyBoard
        result |= inverseBoard & rShift7MyBoard & rShift14MyBoard
        result |= inverseBoard & rShift7MyBoard & lShift7MyBoard

        # check for XX_
        result |= inverseBoard & lShift7MyBoard & lShift14MyBoard

        # check for XX / diagonal
        result |= inverseBoard & lShift8MyBoard & lShift16MyBoard

        result |= inverseBoard & rShift8MyBoard & rShift16MyBoard
        result |= inverseBoard & rShift8MyBoard & rShift16MyBoard
        result |= inverseBoard & rShift8MyBoard & lShift8MyBoard

        # check for XX \ diagonal
        result |= inverseBoard & rShift6MyBoard & rShift12MyBoard
        result |= inverseBoard & rShift6MyBoard & rShift12MyBoard
        result |= inverseBoard & rShift6MyBoard & lShift6MyBoard
        result |= inverseBoard & lShift6MyBoard & lShift12MyBoard

        # check for _XX vertical
        result |= inverseBoard & (myBoard << 1) & (myBoard << 2) \
            & (myBoard << 2)

        return result

    def evaluate1(self, oppBoard, myBoard):
        """
        Returns the number of possible 1 in a rows in bitboard format.
        Running time: O(1)
        Diagonals are skipped since they are worthless.
        """
        inverseBoard = ~(myBoard | oppBoard)
        # check for _X
        result = inverseBoard & (myBoard >> 7)

        # check for X_
        result |= inverseBoard & (myBoard << 7)

        # check for _X vertical
        result |= inverseBoard & (myBoard << 1)

        return result

    def bitboardBits(self, i):
        """"
        Returns the number of bits in a bitboard (7x6).
        Running time: O(1)
        Help from: http://stackoverflow.com/q/9829578/1524592
        """
        i = i & 0xFDFBF7EFDFBF  # magic number to mask to only legal bitboard
        # positions (bits 0-5, 7-12, 14-19, 21-26, 28-33, 35-40, 42-47)
        i = (i & 0x5555555555555555) + ((i & 0xAAAAAAAAAAAAAAAA) >> 1)
        i = (i & 0x3333333333333333) + ((i & 0xCCCCCCCCCCCCCCCC) >> 2)
        i = (i & 0x0F0F0F0F0F0F0F0F) + ((i & 0xF0F0F0F0F0F0F0F0) >> 4)
        i = (i & 0x00FF00FF00FF00FF) + ((i & 0xFF00FF00FF00FF00) >> 8)
        i = (i & 0x0000FFFF0000FFFF) + ((i & 0xFFFF0000FFFF0000) >> 16)
        i = (i & 0x00000000FFFFFFFF) + ((i & 0xFFFFFFFF00000000) >> 32)

        return i

    @lru_cache
    def evaluate(self, oppBoard, myBoard):
        """
        Returns cost of each board configuration.
        winning is a winning move
        blocking is a blocking move
        Running time: O(7n)
        """
        winReward = 9999999
        OppCost3Row = 1000
        MyCost3Row = 3000
        OppCost2Row = 500
        MyCost2Row = 500
        OppCost1Row = 100
        MyCost1Row = 100

        if self.has_won(oppBoard):
            return -winReward
        elif self.has_won(myBoard):
            return winReward
        elif self.has_drawn():
            return 0 # draw score

        get3Win = self.evaluate3(oppBoard, myBoard)
        winning3 = self.bitboardBits(get3Win) * MyCost3Row

        get3Block = self.evaluate3(myBoard, oppBoard)
        blocking3 = self.bitboardBits(get3Block) * -OppCost3Row

        get2Win = self.evaluate2(oppBoard, myBoard)
        winning2 = self.bitboardBits(get2Win) * MyCost2Row

        get2Block = self.evaluate2(myBoard, oppBoard)
        blocking2 = self.bitboardBits(get2Block) * -OppCost2Row

        get1Win = self.evaluate1(oppBoard, myBoard)
        winning1 = self.bitboardBits(get1Win) * MyCost1Row

        get1Block = self.evaluate1(myBoard, oppBoard)
        blocking1 = self.bitboardBits(get1Block) * -OppCost1Row

        return winning3 + blocking3 + winning2 + blocking2\
            + winning1 + blocking1

class Solver(object):
    def __init__(self):
        self.node_count = 0
        self.column_order = []
        for i in range(7):
            self.column_order.append(7 // 2 + (1 - 2 * (i % 2)) * (i + 1) //2)

    # def negamax(self, p, alpha, beta):
    #     self.node_count += 1
    #     if p.has_drawn(): # check for draw game
    #         return 0
        
    #     for x in range(7): # check if current player can win next move
    #         if p.can_play(x) and p.is_winning_move(x):
    #             return 
    #     return alpha

    
    def negamax(self, p:Position, depth, mark, config, alpha=-np.Inf, beta=np.Inf):
        """
        function negamax(node, depth, α, β, color) is
        if depth = 0 or node is a terminal node then
            return color × the heuristic value of node

        childNodes := generateMoves(node)
        childNodes := orderMoves(childNodes)
        value := −∞
        foreach child in childNodes do
            value := max(value, −negamax(child, depth − 1, −β, −α, −color))
            α := max(α, value)
            if α ≥ β then
                break (* cut-off *)
        return alpha
        """
        # p.pretty_print()
        self.node_count += 1
        # Opponent must've won, as the current position was provided from the previous move
        is_terminal = (p.has_drawn() or p.has_won(p.position))
        
        if depth == 0 or is_terminal:
            return -p.evaluate(p.position, p.position ^ p.mask)
        
        valid_moves = [self.column_order[x] for x in range(7) if p.can_play(self.column_order[x])]

        value = -np.Inf
        for col in valid_moves:
            child = Position(p.position, p.mask)
            # print("Playing ", col, "with mark", mark)
            child.play(col)
            # child.pretty_print()

            value = max(value, -self.negamax(child, depth - 1, mark%2+1, config, -beta, -alpha))
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return alpha

    # Uses minimax to calculate value of dropping piece in selected column
    def score_move(self, p:Position, col, mark, config, nsteps):
        next_grid = Position(p.position, p.mask)
        next_grid.play(col)        
        score = -self.negamax(next_grid, nsteps - 1, mark%2+1, config)
        return score

    
    # def solve(self, p, weak=False):
    #     self.node_count = 0
    #     if weak:
    #         return self.negamax(p, -1, 1)
    #     else:
    #         return self.negamax(p, -7 * 6 //2, 7*6//2)

# grid = [
#  [0,0,0,0,0,0,0],
#  [0,0,0,0,0,0,0],
#  [0,0,0,0,0,0,0],
#  [0,0,0,0,0,0,0],
#  [0,0,0,0,0,0,0],
#  [0,0,0,2,1,1,2]
#  ]
    
# position, mask = Position.get_position_mask_bitmap(grid, 1)
# p = Position(position, mask)
# # p = Position(0b0000000000000100000010000000000000000000000000000, 0b0000001000000100000010000001000000000000000000000)
# print("top_mask", format(p.top_mask(1), "064b"))
# print("bottom_mask", format(p.bottom_mask(1), "064b"))
# print("column_mask", format(p.column_mask(1), "064b"))
# print("can play", p.can_play(3))
# p.pretty_print()
# p.play(3)
# print("position after play", format(p.position, "049b"))
# p.pretty_print()
# p.play(3)
# p.play(3)
# p.play(3)
# p.play(3)
# print("can now play", p.can_play(3))
# p.play(2)
# p.pretty_print()
# print("has_drawn", p.has_drawn())

In [17]:
def my_agent(obs, config):
    import numpy as np
    import random
    import time

    

    
    start = time.time()
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # print(grid)
    
    # Very first move, always play the middle
    if sum(obs.board) == 0:
        return 3
    elif sum(obs.board) == 1 and grid[0, 3] == 0:
        return 3 # Play middle whether opponent places there or not

    position, mask = Position.get_position_mask_bitmap(grid, obs.mark)
    # print(position, mask)
    p = Position(position, mask)
    solver = Solver()
    valid_moves = [solver.column_order[x] for x in range(7) if p.can_play(solver.column_order[x])]
    # If there is a winning move, return it now!
    for col in valid_moves:
        if p.is_winning_move(col):
            return col

    # Use the heuristic to assign a score to each possible board in the next step
    if len(valid_moves) >= 6:
        N_STEPS=6
    if len(valid_moves) ==5:
        N_STEPS=8
    if len(valid_moves) ==4:
        N_STEPS= 12
    if len(valid_moves) <=3:
        N_STEPS= 16

    scores = dict(zip(valid_moves, [solver.score_move(p, col, obs.mark, config, N_STEPS) for col in valid_moves]))   
    
    #Get the highest score value    
    
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]    
    # col = random.choice(max_cols) # Taking the center columns first
    col = max_cols[0]
    end = time.time()
    print("my_agent excecution time", (end-start), "move", col, "score", scores[col], "at depth", N_STEPS, "pos count", solver.node_count)
    
    return col

# grid = [[0, 1, 0, 2, 2, 1, 0],
#  [0, 2, 0, 2, 1, 2, 0],
#  [0, 1, 0, 2, 1, 1, 0],
#  [0, 2, 0, 1, 2, 2, 2],
#  [0, 1, 2, 2, 1, 1, 1],
#  [2, 1, 2, 1, 1, 1, 2]]
# grid = [[0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0]]
# grid = [[0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 1, 0, 0, 0],
#  [0, 0, 0, 2, 0, 0, 0],
#  [0, 0, 2, 1, 0, 0, 0]]

# position, mask = Position.get_position_mask_bitmap(grid, 1)
# solver = Solver()
# p = Position(position, mask)
# valid_moves = [solver.column_order[x] for x in range(7) if p.can_play(solver.column_order[x])]
# print(valid_moves)
# scores = dict(zip(valid_moves, [solver.score_move(p, col, 1, 0, 8) for col in valid_moves]))   
# print(scores)
# max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]    
# print(max_cols)
# score = solver.score_move(p, 6, 1, 0, 12)
# print(score)

In [9]:
import inspect
import os

def write_agent_to_file(function, file):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(function))
        print(function, "written to", file)

write_agent_to_file(my_agent, "./submission.py")

<function my_agent at 0x1049c99d0> written to ./submission.py


In [18]:
from kaggle_environments import make, evaluate

# Create the game environment
env = make("connectx", debug=True)

# Two random agents play one game round
# env.run([my_agent, "negamax"])
env.run([my_agent, my_agent3])
# env.run([my_agent3, "negamax"])
# env.run([my_agent, my_agent3])

# Show the game
env.render(mode="ipython")

my_agent excecution time 0.5084433555603027 move 2 score 300 at depth 6 pos count 18805
my_agent3 excecution time 2.124995231628418 move 4 score -90.0 at depth 4 pos count 1216
my_agent excecution time 0.5283079147338867 move 4 score -1900 at depth 6 pos count 20536
my_agent3 excecution time 1.7200658321380615 move 1 score 0.0 at depth 4 pos count 1032
my_agent excecution time 0.28324198722839355 move 4 score -2300 at depth 6 pos count 10840
my_agent3 excecution time 1.364447832107544 move 4 score 0.0 at depth 4 pos count 823
my_agent excecution time 0.2370450496673584 move 3 score -3700 at depth 6 pos count 9106
my_agent3 excecution time 1.3061370849609375 move 3 score -190.0 at depth 4 pos count 816
my_agent excecution time 0.4611198902130127 move 4 score -3100 at depth 6 pos count 14759
my_agent3 excecution time 1.6661059856414795 move 5 score -90.0 at depth 4 pos count 878
my_agent excecution time 0.5133209228515625 move 0 score -2600 at depth 6 pos count 18430
my_agent3 excecution

In [6]:
def get_win_percentages(agent1, agent2, n_rounds=10):
    from kaggle_environments import make, evaluate
    import numpy as np
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # print("outcomes:", outcomes)
    print("outcomes:", outcomes)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("outcomes:", outcomes)
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

In [19]:
get_win_percentages(agent1=my_agent3, agent2=my_agent)

outcomes: [[-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1]]
outcomes: [[-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1]]
Agent 1 Win Percentage: 0.0
Agent 2 Win Percentage: 1.0
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 0
